# Model 5: The Production-Inventory Problem

In this notebook, we will explore a common problem in operations management: how to schedule the production plan of a product over a given time horizon. This problem involves balancing the trade-off between production costs and inventory costs, while satisfying the customer demands as well as production capacity and inventory constraints. We will use linear programming (LP) to formulate and solve this problem, and find the optimal production and inventory levels for each week.

## Modeling components

### Sets

- $T$: Set of time periods (e.g., weeks) representing the planning horizon of our problem.

### Parameters

- $I_0$: The initial inventory, i.e., the amount of the product that is available at the beginning of the first week.
- $h$: Unit holding cost. The unit holding cost is the cost of storing one unit of the product in the inventory for one week. It is assumed that the holding cost per unit is invariant from one time period to another.
- $p_t$: Production capacity in week $t$, $t\in T$. The production capacity is the maximum amount of the product that can be produced in a given week.
- $c_t$: Unit production cost in week $t$, $t\in T$. It indicates the cost of producing one unit of the product in a given week. 
- $d_t$: Demand of week $t$, $t\in T$. It denotes the amount of the product that is required by the customers in a given week. 

### Decision variables

- $x_t$: Amount produced in week $t\in T$
- $s_t$: Inventory level at the end of week $t\in T$

## The linear programming formulation

$$
\begin{array}{rll}
    \min  & \sum_{t\in T} \left(c_t\, x_t + h\, s_t \right) \\[5pt]
    \text{s.t.} & I_0 + x_1 = s_1 + d_1 \\
    & s_{t-1} + x_t = s_t + d_t, & t\in T\setminus\{1\} \\
    & x_t\leq p_t, & t\in T \\[5pt]
    & x_t, s_t \geq 0, & t\in T
\end{array}
$$

## A problem instance

Suppose you are scheduling the weekly production plan of a product for 12 weeks. The plan depends on several parameters, such as the initial inventory, the unit holding cost, the demand, the unit production cost, and the production capacity for each week.
The value of these parameters are provided in the `Data` directory.
The first data file, `Model5_input_data_1.csv`, contains the values of the production cost, the production capacity, and the demand for each week. The file has four columns: `period`, `demand`, `production_cost`, and `production_capacity`.
The second data file, `Model5_input_data_2.csv`, contains the values of the unit holding cost and the initial inventory. The file has two columns: `attribute` and `value`.

Here, we will learn how to read data and store it in a data frame using the `pandas` library in python. First, mount your Google Drive to access the files:

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

Next, we can read the data files from the `data` directory in the shared course folder. Copy the paths to `Model5_input_data_1.csv` and `Model5_input_data_2.csv` files. Next, read the data files as follows:

In [ ]:
import pandas as pd

# read the csv files into a `DataFrame` object
data_periods = pd.read_csv(
    "/content/drive/.../Model5_input_data_1.csv",  # path to your first input data
    index_col="period",  # use the `period` column as the index
)

data_constants = pd.read_csv(
    "/content/drive/.../Model5_input_data_2.csv",  # path to your second input data
)

You can inspect the imported data as follows.

In [ ]:
data_periods

In [ ]:
print(data_constants)

### Define your sets and parameters

In [ ]:
# set of time periods
T = set(data_periods.index)
print(T)

In [ ]:
# create a dictionary to store our constants
keys = data_constants["attribute"]
values = data_constants["value"]
key_values = zip(keys, values)
constants = dict(key_values)

print(constants)

Define a function for easier data access:

In [ ]:
def get_data(param: str, time_period: int = 0):
    match param:
        case "I0":
            return constants["initial_inventory"]
        case "h":
            return constants["holding_cost"]
        case "c":
            return data_periods.loc[time_period, "production_cost"]
        case "p":
            return data_periods.loc[time_period, "production_capacity"]
        case "d":
            return data_periods.loc[time_period, "demand"]

Test the function (optional):

In [ ]:
# get the value of 'h'
h_value = get_data("h")  # no need to provide a time period
print(h_value)

# get the value of 'c' for the second period
c_value = get_data("c", 2)
print(c_value)

## The concrete Pyomo model

### Import required libraries and create the model object

In [ ]:
!pip install gurobipy pyomo

solver_options = {
    "WLSACCESSID": "...",  # your WSL access id (string)
    "WLSSECRET": "...",  # your WSL secret (string)
    "LICENSEID": ...,  # your license id (integer)
}

In [ ]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

mod = pyo.ConcreteModel(name="production_inventory")

#### The decision variables, the objective, and the constraint

In [ ]:
# create dictionaries of decision variables, using index set T:
mod.x = pyo.Var(T, domain=pyo.NonNegativeReals)
mod.s = pyo.Var(T, domain=pyo.NonNegativeReals)

# objective function:
expr = sum(get_data("c", t) * mod.x[t] for t in T)  # production cost
expr += sum(get_data("h") * mod.s[t] for t in T)  # inventory holding cost
mod.obj = pyo.Objective(expr=expr, sense=pyo.minimize)

In [ ]:
# constraints:

# balance constraints
mod.balance = pyo.ConstraintList()  # create an empty list of constraints
for t in T:
    if t == 1:
        # balance constraint for the first period
        mod.balance.add(expr=get_data("I0") + mod.x[t] == mod.s[t] + get_data("d", t))
    else:
        mod.balance.add(expr=mod.s[t - 1] + mod.x[t] == mod.s[t] + get_data("d", t))

# capacity constraints
mod.capacity = pyo.ConstraintList()
for t in T:
    mod.capacity.add(expr=mod.x[t] <= get_data("p", t))

Inspect your created objects (optional):

In [ ]:
mod.obj.pprint()
mod.balance.pprint()
mod.capacity.pprint()

### Solve the model

In [ ]:
# remove the options argument if not using WLS
opt = SolverFactory("gurobi", solver_io="python", manage_env=True, options=solver_options)
result = opt.solve(mod, tee=False)

### Display and interpret the results

In [ ]:
# display calculation time
print("Solution time = {:.2f}s".format(result.solver.wallclock_time))

# display the total cost
print("Total cost = ${:,.2f}".format(pyo.value(mod.obj)))

# display demand, production, inventory, and unused capacity for each period
for t in T:
    print("Week", t, end=": ")
    print("demand=", get_data("d", t), end=", ")
    print("production=", pyo.value(mod.x[t]), end=", ")
    print("inventory=", pyo.value(mod.s[t]), end=", ")
    print("unused capacity=", get_data("p", t) - pyo.value(mod.x[t]))

#### Exercise 1: Save the results in a data frame

In [ ]:
solution_data = {
    "period": [],
    "demand": [],
    "production": [],
    "inventory": [],
    "unused_capacity": [],
}

for t in T:
    solution_data["period"].append(t)
    solution_data["demand"].append(get_data("d", t))
    solution_data["production"].append(pyo.value(mod.x[t]))
    solution_data["inventory"].append(pyo.value(mod.s[t]))
    solution_data["unused_capacity"].append(get_data("p", t) - pyo.value(mod.x[t]))

solution_data = pd.DataFrame(solution_data)
# set the index
solution_data.set_index("period", inplace=True)

solution_data

#### Exercise 2: Plot production and inventory levels

In [ ]:
# Install the required libraries if not already installed:
!pip install matplotlib seaborn

In [ ]:
# Import plotting libraries
from matplotlib import pyplot as plt
import seaborn as sns

# plot production and inventory values in the solution_data DataFrame
# using the lineplot function from seaborn:
sns.lineplot(data=solution_data[["production", "inventory"]], markers=True)

# set axis labels
plt.xlabel("Week")
plt.ylabel("Level")

plt.show()